# Notebook: Train Model

## Packages

In [1]:
from load_dataset_folds import load_dataset_folds
from transformers import set_seed
from ACD import train_ACD_model
from E2E import train_E2E_model
from OTE import train_OTE_model
from ACSA import train_ACSA_model
from TASD import train_TASD_model
import pandas as pd
import numpy as np
import constants
import warnings
import shutil
import torch
import json
import sys

## Parameters

In [2]:
LLM_NAME = "Llama13B"
N_REAL = 500
N_SYNTH = 0
TARGET = "end_2_end_absa" # "aspect_category", "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"
LLM_SAMPLING = "random"

## Settings

In [3]:
# Set seeds
torch.device(constants.DEVICE)
torch.manual_seed(constants.RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(constants.RANDOM_SEED)
set_seed(constants.RANDOM_SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(constants.RANDOM_SEED)

# Ignore warnings
warnings.filterwarnings("ignore", category=FutureWarning,
                        module="transformers.optimization")

# Disable Pycache
sys.dont_write_bytecode = True

## Code

In [4]:
train_dataset, test_dataset = load_dataset_folds(LLM_NAME, N_REAL, N_SYNTH, LLM_SAMPLING)

500 5
500 5


### Load Model

In [5]:
if TARGET == "aspect_category":
    results = train_ACD_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [6]:
if TARGET == "aspect_category_sentiment":
    results = train_ACSA_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [7]:
if TARGET == "end_2_end_absa":
    results = train_E2E_model(LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

527


Some weights of BertForSpanCategorizationE2E were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSpanCategorizationE2E were not initialized from the model checkpoint at deepset/gbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/23 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.5093, 'learning_rate': 0.0, 'epoch': 1.0}


  0%|          | 0/22 [00:00<?, ?it/s]

527
{'eval_loss': 0.37403249740600586, 'eval_tp_POSITIVE': 2, 'eval_tn_POSITIVE': 0, 'eval_fp_POSITIVE': 5403, 'eval_fn_POSITIVE': 298, 'eval_accuracy_POSITIVE': 0.0003506926179203928, 'eval_precision_POSITIVE': 0.0003700277520814061, 'eval_recall_POSITIVE': 0.006666666666666667, 'eval_f1_POSITIVE': 0.0007011393514460998, 'eval_tp_NEUTRAL': 0, 'eval_tn_NEUTRAL': 0, 'eval_fp_NEUTRAL': 188, 'eval_fn_NEUTRAL': 17, 'eval_accuracy_NEUTRAL': 0.0, 'eval_precision_NEUTRAL': 0.0, 'eval_recall_NEUTRAL': 0.0, 'eval_f1_NEUTRAL': 0, 'eval_tp_NEGATIVE': 6, 'eval_tn_NEGATIVE': 0, 'eval_fp_NEGATIVE': 2191, 'eval_fn_NEGATIVE': 204, 'eval_accuracy_NEGATIVE': 0.0024989587671803417, 'eval_precision_NEGATIVE': 0.0027309968138370506, 'eval_recall_NEGATIVE': 0.02857142857142857, 'eval_f1_NEGATIVE': 0.004985459077690072, 'eval_f1_micro': 0.0019237705903571002, 'eval_f1_macro': 0.001895532809712057, 'eval_tp': 8, 'eval_tn': 0, 'eval_fp': 7782, 'eval_fn': 519, 'eval_accuracy': 0.0009628114093152004, 'eval_preci

  0%|          | 0/22 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
if TARGET == "aspect_term":
    results = train_OTE_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

In [ ]:
if TARGET == "target_aspect_sentiment_detection":
    results = train_TASD_model(
        LLM_NAME, N_REAL, N_SYNTH, TARGET, LLM_SAMPLING, train_dataset, test_dataset)

### Save Results

In [ ]:
with open(f'results_json/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.json', 'w') as json_file:
    json.dump(results, json_file)

df = pd.DataFrame([results])
df.to_csv(f'results_csv/results_{LLM_NAME}_real{N_REAL}_synth{N_SYNTH}_{TARGET}_{LLM_SAMPLING}.csv', index=False)

In [ ]:
results

### Remove useless folders

In [ ]:
try:
    shutil.rmtree("outputs")
except:
    pass